# League Dash Player Stats

This endpoint allows retrieval of all player stats within a season, which makes calling multiple seasons more feasible

In [2]:
import json
import numpy as np
import pickle
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.static import players

In [17]:
league_dash = leaguedashplayerstats.LeagueDashPlayerStats(
    measure_type_detailed_defense="Base",
    season_type_all_star="Regular Season",
    season="2001-02",
    # must set to "N"; otherwise weird negatives appear
    # also, p_m is included regardless of this setting.
    plus_minus="N",
    per_mode_detailed="Per36",
)

In [18]:
league_dash.get_request_url()

'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=&PaceAdjust=N&PerMode=Per36&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2001-02&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=&TwoWay=&VsConference=&VsDivision=&Weight='

In [20]:
df_list = league_dash.get_data_frames()
df_list[0]

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,CFID,CFPARAMS
0,2062,A.J. Guyton,A.J.,1610612741,CHI,24.0,45,10,35,0.222,...,433,153,138,271,304,225,18,193,5,"2062,1610612741"
1,243,Aaron McKie,Aaron,1610612755,PHI,29.0,48,26,22,0.542,...,415,153,144,84,177,176,18,152,5,"243,1610612755"
2,1425,Aaron Williams,Aaron,1610612751,NJN,30.0,82,52,30,0.634,...,72,153,165,82,123,225,18,146,5,"1425,1610612751"
3,1502,Adonal Foyle,Adonal,1610612744,GSW,27.0,79,19,60,0.241,...,125,96,365,353,56,127,18,112,5,"1502,1610612744"
4,1559,Adrian Griffin,Adrian,1610612742,DAL,27.0,58,40,18,0.690,...,185,153,291,53,243,225,18,292,5,"1559,1610612742"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,1895,William Avery,William,1610612750,MIN,22.0,28,17,11,0.607,...,351,153,341,277,392,225,18,382,5,"1895,1610612750"
436,2216,Zach Randolph,Zach,1610612757,POR,20.0,41,22,19,0.537,...,117,153,60,284,51,225,18,46,5,"2216,1610612757"
437,1442,Zeljko Rebraca,Zeljko,1610612765,DET,30.0,74,46,28,0.622,...,38,76,99,116,58,176,18,68,5,"1442,1610612765"
438,1985,Zendon Hamilton,Zendon,1610612743,DEN,27.0,54,17,37,0.315,...,114,27,164,405,159,96,18,143,5,"1985,1610612743"


In [21]:
df = df_list[0]
df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
       'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
       'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
       'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
       'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
       'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK', 'CFID', 'CFPARAMS'],
      dtype='object')

In [23]:
drop_stats = [
    "W",
    "L",
    "W_PCT",
    "FG_PCT",
    "FG3_PCT",
    "FT_PCT",
    "REB",
    "NBA_FANTASY_PTS",
    "DD2",
    "TD3",
    "WNBA_FANTASY_PTS",
]
drop_ranks = [stat + "_RANK" for stat in drop_stats]
drop_cols = [
    "NICKNAME",
    "TEAM_ID",
    *drop_stats,
    *drop_ranks,
    "CFID",
    "CFPARAMS",
    "W",
    "L",
    "W_PCT",
]
df.drop(drop_cols, axis=1).sort_values(by="MIN", ascending=False).head(10)

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,AGE,GP,MIN,FGM,FGA,FG3M,FG3A,...,DREB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
24,952,Antoine Walker,BOS,25.0,81,3406.435000,7.0,17.8,2.3,6.8,...,84,105,78,138,262,46,333,153,32,95
395,1495,Tim Duncan,SAS,26.0,82,3331.660000,8.3,16.3,0.0,0.1,...,12,137,56,366,33,67,352,153,9,22
258,84,Latrell Sprewell,NYK,31.0,81,3327.863333,6.2,15.4,1.6,4.4,...,382,132,118,241,367,206,420,153,67,273
33,1884,Baron Davis,CHH,23.0,82,3319.765000,6.1,14.5,1.8,5.2,...,353,19,77,36,219,255,321,153,79,123
320,1718,Paul Pierce,BOS,24.0,82,3298.856667,7.7,17.4,2.3,5.7,...,134,162,80,62,135,100,326,153,7,62
158,56,Gary Payton,SEA,33.0,82,3297.143333,8.0,17.2,0.8,2.6,...,284,13,151,103,305,222,409,152,20,106
380,950,Stephon Marbury,PHX,25.0,82,3186.473333,7.1,16.0,0.8,2.8,...,411,20,31,293,369,105,394,151,33,227
245,708,Kevin Garnett,MIN,26.0,81,3171.478333,7.5,15.9,0.4,1.3,...,11,85,86,216,81,274,397,70,23,44
189,147,Jalen Rose,CHI,29.0,83,3155.568333,7.6,16.6,1.0,2.8,...,260,110,147,285,217,250,284,100,25,239
135,224,Eddie Jones,MIA,30.0,81,3155.218333,5.9,13.7,1.7,4.4,...,271,149,301,122,146,363,275,150,68,152


Get both traditional *and* advanced stats, *and* multiple seasons?

In [3]:
def get_leaguedash(
    measure_type: str = "Base",
    season_type: str = "Regular Season",
    season: str = "2001-02",
    per_mode: str = "Per36",
    df: bool = True,
):
    """Wrapper for league dash player stats request"""

    league_dash = leaguedashplayerstats.LeagueDashPlayerStats(
        measure_type_detailed_defense=measure_type,
        season_type_all_star=season_type,
        season=season,
        plus_minus="N",
        per_mode_detailed=per_mode,
    )
    if df:
        res = league_dash.get_data_frames()[0]
    else:
        res = json.loads(league_dash.get_normalized_json())

    return res

In [4]:
measure_type = "|".join(["Base", "Advanced"])
season_type = "|".join(["Regular Season", "Playoffs"])
seasons = "|".join(["2005-06", "2006-07"])

# res = get_leaguedash(measure_type, season_type, seasons)
# res = get_leaguedash(measure_type=measure_type)


# res = get_leaguedash(season=seasons)
res_df = get_leaguedash()

No bueno. One type, one season, one season type at a time

In [14]:
res_df.drop(drop_cols, axis=1).sort_values("MIN", ascending=False).head(10)

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,AGE,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS
24,952,Antoine Walker,BOS,25.0,81,3406.435000,0.7,1.8,-0.001,1.6,...,-2.7,-0.5,-2.3,1.7,-1.6,1.6,0.1,-0.1,2.5,2.5
395,1495,Tim Duncan,SAS,26.0,82,3331.660000,0.7,-3.7,0.039,0.8,...,0.3,1.3,-0.1,-0.4,1.2,-1.2,-3.3,3.3,6.1,6.1
258,84,Latrell Sprewell,NYK,31.0,81,3327.863333,-0.9,0.0,-0.014,0.3,...,-1.7,-0.1,0.5,-0.8,-0.8,0.8,1.0,-1.0,-2.6,-2.6
33,1884,Baron Davis,CHH,23.0,82,3319.765000,0.7,0.7,0.006,-0.5,...,2.3,1.3,-0.1,0.3,0.4,-0.4,0.6,-0.6,1.6,1.6
320,1718,Paul Pierce,BOS,24.0,82,3298.856667,1.0,1.5,0.005,1.6,...,-2.6,-0.5,-2.4,1.6,-1.5,1.5,0.0,0.0,3.4,3.4
158,56,Gary Payton,SEA,33.0,82,3297.143333,1.5,0.7,0.019,0.3,...,-1.5,0.7,-1.3,1.1,-0.4,0.4,1.1,-1.1,2.2,2.2
380,950,Stephon Marbury,PHX,25.0,82,3186.473333,0.9,2.6,-0.004,-1.0,...,-0.7,0.0,-0.6,-0.1,-0.8,0.8,2.3,-2.3,-1.1,-1.1
245,708,Kevin Garnett,MIN,26.0,81,3171.478333,2.3,1.9,0.024,-1.1,...,4.8,2.0,0.1,-0.3,0.6,-0.6,-0.1,0.1,4.1,4.1
189,147,Jalen Rose,CHI,29.0,83,3155.568333,0.0,-0.6,0.005,-0.1,...,-1.6,0.0,0.1,0.0,0.3,-0.3,1.3,-1.3,-1.4,-1.4
135,224,Eddie Jones,MIA,30.0,81,3155.218333,2.0,1.1,0.027,-0.5,...,0.9,2.6,0.4,-0.4,1.2,-1.2,1.0,-1.0,0.8,0.8


In [25]:
res_json = league_dash.get_normalized_dict()
print(f"res_json type: {type(res_json)}\nkeys: {list(res_json.keys())}")

res_json type: <class 'dict'>
keys: ['LeagueDashPlayerStats']


In [26]:
print(
    f"dict value type: {type(res_json['LeagueDashPlayerStats'])}\nlen: {len(res_json['LeagueDashPlayerStats'])}"
)

dict value type: <class 'list'>
len: 440


In [27]:
sample_record = res_json["LeagueDashPlayerStats"][0]
print(f"type: {type(sample_record)}\ncontent: {sample_record}")

type: <class 'dict'>
content: {'PLAYER_ID': 2062, 'PLAYER_NAME': 'A.J. Guyton', 'NICKNAME': 'A.J.', 'TEAM_ID': 1610612741, 'TEAM_ABBREVIATION': 'CHI', 'AGE': 24.0, 'GP': 45, 'W': 10, 'L': 35, 'W_PCT': 0.222, 'MIN': 603.93, 'FGM': 5.2, 'FGA': 14.5, 'FG_PCT': 0.361, 'FG3M': 2.7, 'FG3A': 7.3, 'FG3_PCT': 0.374, 'FTM': 1.3, 'FTA': 1.6, 'FT_PCT': 0.815, 'OREB': 0.7, 'DREB': 1.9, 'REB': 2.6, 'AST': 4.8, 'TOV': 2.2, 'STL': 0.6, 'BLK': 0.4, 'BLKA': 0.2, 'PF': 1.4, 'PFD': 0.0, 'PTS': 14.5, 'PLUS_MINUS': -2.6, 'NBA_FANTASY_PTS': 25.8, 'DD2': 0, 'TD3': 0, 'WNBA_FANTASY_PTS': 26.8, 'GP_RANK': 296, 'W_RANK': 360, 'L_RANK': 129, 'W_PCT_RANK': 416, 'MIN_RANK': 301, 'FGM_RANK': 159, 'FGA_RANK': 73, 'FG_PCT_RANK': 385, 'FG3M_RANK': 7, 'FG3A_RANK': 7, 'FG3_PCT_RANK': 90, 'FTM_RANK': 349, 'FTA_RANK': 379, 'FT_PCT_RANK': 100, 'OREB_RANK': 347, 'DREB_RANK': 422, 'REB_RANK': 418, 'AST_RANK': 83, 'TOV_RANK': 168, 'STL_RANK': 385, 'BLK_RANK': 251, 'BLKA_RANK': 420, 'PF_RANK': 433, 'PFD_RANK': 153, 'PTS_RANK': 

In [31]:
some_recs = res_json["LeagueDashPlayerStats"][:5]
df_some_recs = pd.DataFrame.from_dict(
    some_recs,
    orient="columns",
).set_index("PLAYER_ID", drop=True)
df_some_recs

,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,CFID,CFPARAMS
PLAYER_ID,,,,,,,,,,,,,,,,,,,,,
2062,A.J. Guyton,A.J.,1610612741,CHI,24.0,45,10,35,0.222,603.930000,...,433,153,138,271,304,225,18,193,5,"2062,1610612741"
243,Aaron McKie,Aaron,1610612755,PHI,29.0,48,26,22,0.542,1469.013333,...,415,153,144,84,177,176,18,152,5,"243,1610612755"
1425,Aaron Williams,Aaron,1610612751,NJN,30.0,82,52,30,0.634,1549.918333,...,72,153,165,82,123,225,18,146,5,"1425,1610612751"
1502,Adonal Foyle,Adonal,1610612744,GSW,27.0,79,19,60,0.241,1485.290000,...,125,96,365,353,56,127,18,112,5,"1502,1610612744"
1559,Adrian Griffin,Adrian,1610612742,DAL,27.0,58,40,18,0.690,1386.620000,...,185,153,291,53,243,225,18,292,5,"1559,1610612742"
